In [24]:
%set_env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [25]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)

add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool =  FunctionTool.from_defaults(fn=mystery)

In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI()
response =  llm.predict_and_call(
    [add_tool,mystery_tool],
    "tell me output of mystery function on 10 and 20"
)


In [6]:
print(response)

900


In [7]:
#RAG Tool

In [8]:
from llama_index.core import SimpleDirectoryReader
documents =  SimpleDirectoryReader(input_files=['budgetpdf.pdf']).load_data()

In [9]:
from llama_index.core.node_parser import SentenceSplitter

splitter =  SentenceSplitter(chunk_size=1024)
nodes= splitter.get_nodes_from_documents(documents)

In [10]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: budgetpdf.pdf
file_path: budgetpdf.pdf
file_type: application/pdf
file_size: 1585381
creation_date: 2024-06-05
last_modified_date: 2024-05-01

PRESS INFORMATION BUREAU 
GOVERNMENT OF INDIA 
**** 
New Delhi, 1st February, 2024 
 
HIGHLIGHTS OF THE INTERIM UNION BUDGET 2024-25 
 
With the ‘mantra’ of ‘Sabka Saath, Sabka Vikas, and Sabka Vishwas’  and the whole of nation 
approach of “Sabka Prayas” , the Union Minister for Finance and Corporate Affairs Smt. 
Nirmala Sitharaman presented the Interim Union Budget 2024-25 in Parliament, today. The key 
highlights of the Budget are as follows: 
 
Part A 
 
Social Justice 
 Prime Minister to focus on upliftment of four major castes, that is, ‘Garib’ (Poor), 
‘Mahilayen’ (Women), ‘Yuva’ (Youth) and  ‘Annadata’(Farmer) . 
 
‘Garib Kalyan, Desh ka Kalyan’ 
 Government assisted 25 crore people out of multi-dimensional poverty in last 10 years. 
 DBT of Rs. 34 lakh crore using PM-Jan Dhan accounts led to savings of Rs. 2

In [11]:
print(nodes[5].get_content(metadata_mode="all"))

page_label: 6
file_name: budgetpdf.pdf
file_path: budgetpdf.pdf
file_type: application/pdf
file_size: 1585381
creation_date: 2024-06-05
last_modified_date: 2024-05-01

PRESS INFORMATION BUREAU 
GOVERNMENT OF INDIA 
*** 
INDIA’S REAL GDP PROJECTED TO GROW AT 7.3 PER CENT IN FY 2023-24 
FINANCE AND CORPORATE AFFAIRS MINISTER SMT NIRMALA SITHARAMAN 
PRESENTS THE INTERIM UNION BUDGET FOR 2024-2025 IN PARLIAMENT  
CAPITAL EXPENDITURE OUTLAY FOR THE NEXT YEAR IS BEING INCREASED 
BY 11.1 PER CENT TO RS 11,11,111 CRORE, WHICH WOULD BE  3.4 PER CENT OF 
THE GDP 
                FISCAL DEFICIT IN 2024-25 IS ESTIMATED TO BE 5.1 PER CENT OF GDP 
FDI INFLOW DURING 2014-23 WAS USD 596 BILLION, WHICH IS TWICE THE 
INFLOW DURING 2005-14. 
UPLIFTMENT OF ‘GARIB’ (POOR), ‘MAHILAYEN’ (WOMEN),  ‘YUVA’ (YOUTH) 
AND  ‘ANNADATA’ (FARMER) IS THE HIGHEST PRIORITY OF GOVERNMENT    
A CORPUS OF RUPEES ONE LAKH CRORE WILL BE ESTABLISHED WITH 
FIFTY-YEAR INTEREST FREE LOAN FOR YOUTH 
SCHEME OF 50 YEAR INTEREST FREE

In [12]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=3)

In [13]:
query_engine.query("what are Momentum for Nari Shakti")

Response(response="Empowerment of women through entrepreneurship, ease of living, and dignity for them has gained momentum. This includes initiatives such as providing loans through the Mudra Yojana to women entrepreneurs, increasing female enrollment in higher education, promoting STEM courses for girls and women, increasing participation of women in the workforce, making 'triple talaq' illegal, and reserving one-third of seats for women in the Lok Sabha and State Legislative Assemblies. Additionally, over 70% of houses under the PM Awas Yojana in rural areas have been given to women as sole or joint owners.", source_nodes=[NodeWithScore(node=TextNode(id_='2cec13bd-50e6-4cac-a3b3-8246e1e66f97', embedding=None, metadata={'page_label': '48', 'file_name': 'budgetpdf.pdf', 'file_path': 'budgetpdf.pdf', 'file_type': 'application/pdf', 'file_size': 1585381, 'creation_date': '2024-06-05', 'last_modified_date': '2024-05-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size'

In [14]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

In [17]:
response = query_engine.query(
    "what are Momentum for Nari Shakti in page 0?", 
)

In [18]:
print(response)

Over 70% houses under PM Awas Yojana given to women from rural areas.


In [19]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [21]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "what are Momentum for Nari Shakti in page 1?",
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "Momentum for Nari Shakti", "page_numbers": ["1"]}
=== Function Output ===
30 crore Mudra Yojana loans have been provided to women entrepreneurs. Female enrolment in higher education has increased by 28%. In STEM courses, girls and women make up 43% of enrolment, which is one of the highest rates globally.


In [22]:
llm.predict_and_call(
    [vector_query_tool, add_tool,mystery_tool], 
    "what are Momentum for Nari Shakti in page 1?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "Momentum for Nari Shakti", "page_numbers": ["1"]}
=== Function Output ===
The momentum for Nari Shakti includes providing 30 crore Mudra Yojana loans to women entrepreneurs, witnessing a 28% increase in female enrollment in higher education, and having girls and women constitute 43% of enrollment in STEM courses, which is one of the highest rates globally.


AgentChatResponse(response='The momentum for Nari Shakti includes providing 30 crore Mudra Yojana loans to women entrepreneurs, witnessing a 28% increase in female enrollment in higher education, and having girls and women constitute 43% of enrollment in STEM courses, which is one of the highest rates globally.', sources=[ToolOutput(content='The momentum for Nari Shakti includes providing 30 crore Mudra Yojana loans to women entrepreneurs, witnessing a 28% increase in female enrollment in higher education, and having girls and women constitute 43% of enrollment in STEM courses, which is one of the highest rates globally.', tool_name='vector_tool', raw_input={'args': (), 'kwargs': {'query': 'Momentum for Nari Shakti', 'page_numbers': ['1']}}, raw_output=Response(response='The momentum for Nari Shakti includes providing 30 crore Mudra Yojana loans to women entrepreneurs, witnessing a 28% increase in female enrollment in higher education, and having girls and women constitute 43% of enrol

In [23]:
llm.predict_and_call(
    [vector_query_tool, add_tool,mystery_tool], 
    "what is 10+2", 
    verbose=True
)

=== Calling Function ===
Calling function: add with args: {"x": 10, "y": 2}
=== Function Output ===
12


AgentChatResponse(response='12', sources=[ToolOutput(content='12', tool_name='add', raw_input={'args': (), 'kwargs': {'x': 10, 'y': 2}}, raw_output=12, is_error=False)], source_nodes=[], is_dummy_stream=False)